In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import sys, os, pathlib, shutil, platform
import pandas as pd

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [ ]:
def freight_aggregation(crt_year, metric_column, analysis_type, regional_flag='regional', non_jets_flag='nonjets'):
    """
    For each year in the years_list passed to the function, the sum of freight for each month's flights that year is computed and saved in a csv file.
    Thus 12 monthly values are calculated for each analysis type.
    
    Note that this is made for non-jets only, as indicated by using only 'AIRCRAFT_GROUP' codes <= 5. Jets have codes >= 6. 
    Additionally, this is made for regional flights, which are those with nautical mile distances b/w 75-500.
    
    Analysis 0: origin US, dest US
    Analysis 1: origin US, dest non-US
    Analysis 2: origin non-US, dest US
    Analysis 3: origin all, dest all
    Analysis 4: origin all states, dest all
    Analysis 5: origin all, dest all states
    Analysis 6: origin US, dest all
    Analysis 7: origin AFW, destination all 
    Analysis 8: origin all, destination AFW
    Analysis 9: origin KIS, destination all
    Analysis 10: origin all, destination KIS
AFW (Perot Field Fort Worth Alliance Airport) is an airport located in Fort Worth, Texas. 
KIS (Visalia Municipal Airport) is an airport located in Visalia, California.
Both are airports of interest to the PAAV team.
    
    # cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 10194)]
# cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 10194)]

    """

    cleaned_data = pd.read_csv(str('./../../../data/paav_cargo/T-100Segment_AllCarriers/DL_SelectFields_AllMonths_AllGeo_'+
                 str(crt_year)
                 +'/T_T100_SEGMENT_ALL_CARRIER.csv'))

    cleaned_data.drop(cleaned_data[cleaned_data['DISTANCE'] == 0].index, inplace = True)
    cleaned_data.dropna(subset=['ORIGIN_STATE_NM'], inplace = True)
    cleaned_data.dropna(subset=['DEST_STATE_NM'], inplace = True)
    cleaned_data.dropna(subset=['ORIGIN_COUNTRY'], inplace = True)
    cleaned_data.dropna(subset=['DEST_COUNTRY'], inplace = True)
    cleaned_data.drop(cleaned_data[cleaned_data['DEPARTURES_PERFORMED'] == 0].index, inplace = True)
    cleaned_data['DISTANCE'] = cleaned_data['DISTANCE'].apply(lambda x: x*0.868976) #convert miles to nautical miles

    cleaned_data['ds'] = cleaned_data['YEAR'].map(str) +"-"+ cleaned_data["MONTH"].map(str) + "-01"
    cleaned_data['ds'] = pd.to_datetime(cleaned_data['ds'],format='%Y-%m-%d')
    # cleaned_data.sort_values(by='ds', ascending=True, inplace = True)

    if regional_flag=='regional':
        #specific to regionality as mentioned in function desc
        cleaned_data = cleaned_data[(cleaned_data['DISTANCE'] <= 500) & (cleaned_data['DISTANCE'] >= 75)].reset_index()
        cleaned_data.drop(['index'], axis=1, inplace = True)

    if non_jets_flag=='nonjets':
        #specific to nonjets as mentioned in function desc
        cleaned_data = (cleaned_data[cleaned_data['AIRCRAFT_GROUP'] <= 5]).reset_index()
        cleaned_data.drop(['index'], axis=1, inplace = True)


    if analysis_type == 0:
        print(crt_year)
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US') & (cleaned_data['DEST_COUNTRY'] =='US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'US'


    elif analysis_type == 1:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US') & (cleaned_data['DEST_COUNTRY'] != 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'non-US'


    elif analysis_type == 2:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] != 'US') & (cleaned_data['DEST_COUNTRY'] == 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'non-US'
        cleaned_data['dest'] = 'US'


    elif analysis_type == 3:
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'all'


#     elif analysis_type == 4:
#         allstates = [None] * cleaned_data['ORIGIN_STATE_NM'].nunique()
#         allstates_index = 0
#         for crt_state in cleaned_data['ORIGIN_STATE_NM'].unique():
#             crtdata = cleaned_data[(cleaned_data['ORIGIN_STATE_NM'] == crt_state)].groupby('ds')['FREIGHT'].agg(['sum','count'])
#             crtdata.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
#             crtdata['origin'] = crt_state
#             crtdata['dest'] = 'all'
#             allstates[allstates_index] = crtdata
#             allstates_index=allstates_index+1

#         crtdata = pd.concat(allstates)
#         cleaned_data = crtdata.reset_index()
#         # cleaned_data.drop(['index'], axis=1, inplace = True)

    elif analysis_type == 4:
        cleaned_data= cleaned_data.groupby(['ds', 'ORIGIN_STATE_NM'])['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights', 'ORIGIN_STATE_NM': 'origin'}, inplace = True)
        cleaned_data['dest'] = 'all'
        
    elif analysis_type == 5:
        cleaned_data= cleaned_data.groupby(['ds', 'DEST_STATE_NM'])['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights', 'DEST_STATE_NM': 'dest'}, inplace = True)
        cleaned_data['origin'] = 'all'
        
    elif analysis_type == 6:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'all'
        
        
    elif analysis_type == 7:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 10194)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'AFW'
        cleaned_data['dest'] = 'all'
        
        
    elif analysis_type == 8:
        cleaned_data = cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 10194)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'AFW'
        
        
    elif analysis_type == 9:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 15601)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'KIS'
        cleaned_data['dest'] = 'all' 
        
    elif analysis_type == 10:
        cleaned_data = cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 15601)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'KIS' 
      

    cleaned_data.to_csv('./../../../data/paav_cargo/agg_data/freight_aggregation__' +
                            str(crt_year) + '_analysis'+ str(analysis_type) + '_' + regional_flag+ '_' + non_jets_flag +
                        '.csv', mode='a', index=False, header=True)

In [ ]:
years_list = [yr for yr in range (1990,2024)]

for analysis_type in range(0,11):
    for regional_flag in [ 'alldistance' , 'regional']:
        for non_jets_flag in ['jetsandnonjets', 'nonjets']: #need to add jets
            for crt_year in years_list:
                data_file = './../../../data/paav_cargo/agg_data/freight_aggregation__' + \
                                    str(crt_year) + '_analysis'+ str(analysis_type)+ '_' + regional_flag+ '_' + non_jets_flag +'.csv'
                if os.path.exists(data_file): os.remove(data_file), print("Deleted file " +data_file + ".")
                else: print("The file " +data_file + " does not exist")

                freight_aggregation(crt_year, 'FREIGHT', analysis_type, regional_flag, non_jets_flag)

In [ ]:
# def spike_investigation_2014_analysis9(crt_year):
#     cleaned_data = pd.read_csv(str('./../../../data/paav_cargo/T-100Segment_AllCarriers/DL_SelectFields_AllMonths_AllGeo_'+
#                  str(crt_year)
#                  +'/T_T100_SEGMENT_ALL_CARRIER.csv'))

#     cleaned_data.drop(cleaned_data[cleaned_data['DISTANCE'] == 0].index, inplace = True)
#     cleaned_data.dropna(subset=['ORIGIN_STATE_NM'], inplace = True)
#     cleaned_data.dropna(subset=['DEST_STATE_NM'], inplace = True)
#     cleaned_data.dropna(subset=['ORIGIN_COUNTRY'], inplace = True)
#     cleaned_data.dropna(subset=['DEST_COUNTRY'], inplace = True)
#     cleaned_data.drop(cleaned_data[cleaned_data['DEPARTURES_PERFORMED'] == 0].index, inplace = True)
#     cleaned_data['DISTANCE'] = cleaned_data['DISTANCE'].apply(lambda x: x*0.868976) #convert miles to nautical miles

#     cleaned_data['ds'] = cleaned_data['YEAR'].map(str) +"-"+ cleaned_data["MONTH"].map(str) + "-01"
#     cleaned_data['ds'] = pd.to_datetime(cleaned_data['ds'],format='%Y-%m-%d')
#     # cleaned_data.sort_values(by='ds', ascending=True, inplace = True)

#     if regional_flag=='regional':
#         #specific to regionality as mentioned in function desc
#         cleaned_data = cleaned_data[(cleaned_data['DISTANCE'] <= 500) & (cleaned_data['DISTANCE'] >= 75)].reset_index()
#         cleaned_data.drop(['index'], axis=1, inplace = True)

#     if non_jets_flag=='nonjets':
#         #specific to nonjets as mentioned in function desc
#         cleaned_data = (cleaned_data[cleaned_data['AIRCRAFT_GROUP'] <= 5]).reset_index()
#         cleaned_data.drop(['index'], axis=1, inplace = True)
        
#     cleaned_data = cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 15601)]
#     # cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
#     # cleaned_data.reset_index(inplace=True)
#     # cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
#     cleaned_data=cleaned_data[['AIRLINE_ID', 'YEAR', 'FREIGHT', 'ds']]
#     cleaned_data['origin'] = 'KIS'
#     cleaned_data['dest'] = 'all'
    
#     cleaned_data.to_csv('./../../../data/paav_cargo/2014_boom_analysis9_investigation/2014_boom_analysis9_investigation' +
#                             str(crt_year) + '_analysis'+ str(9) + '_' + regional_flag+ '_' + non_jets_flag +
#                         '.csv', mode='a', index=False, header=True)

In [ ]:
# for crt_year in [yr for yr in range(2002,2024)]:
#     spike_investigation_2014_analysis9(crt_year)

This is the improved function, which now calculates monthly sums not only for nonjets and jetsandnonjets, but now just uniquely jets too!

In [ ]:
def freight_aggregation(crt_year, metric_column, analysis_type, regional_flag, aircraft_flag):
    """
    For each year in the years_list passed to the function, the sum of freight for each month's flights that year is computed and saved in a csv file.
    Thus 12 monthly values are calculated for each analysis type.
    
    This is made for non-jets, jets, and both. We use 'AIRCRAFT_GROUP' codes to distinguish between them: <= 5 are nonjets, jets have codes >= 6. 
    Additionally, this is made for regional flights, which are those with nautical mile distances b/w 75-500.
    
    Analysis 0: origin US, dest US
    Analysis 1: origin US, dest non-US
    Analysis 2: origin non-US, dest US
    Analysis 3: origin all, dest all
    Analysis 4: origin all states, dest all
    Analysis 5: origin all, dest all states
    Analysis 6: origin US, dest all
    Analysis 7: origin AFW, destination all 
    Analysis 8: origin all, destination AFW
    Analysis 9: origin KIS, destination all
    Analysis 10: origin all, destination KIS
AFW (Perot Field Fort Worth Alliance Airport) is an airport located in Fort Worth, Texas. 
KIS (Visalia Municipal Airport) is an airport located in Visalia, California.
Both are airports of interest to the PAAV team.

aircraft_flag_options = ['nonjets', 'jets', 'jetsandnonjets']
    
# cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 10194)]
# cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 10194)]

    """

    cleaned_data = pd.read_csv(str('./../../../data/paav_cargo/T-100Segment_AllCarriers/DL_SelectFields_AllMonths_AllGeo_'+
                 str(crt_year)
                 +'/T_T100_SEGMENT_ALL_CARRIER.csv'))

    cleaned_data.drop(cleaned_data[cleaned_data['DISTANCE'] == 0].index, inplace = True)
    cleaned_data.dropna(subset=['ORIGIN_STATE_NM'], inplace = True)
    cleaned_data.dropna(subset=['DEST_STATE_NM'], inplace = True)
    cleaned_data.dropna(subset=['ORIGIN_COUNTRY'], inplace = True)
    cleaned_data.dropna(subset=['DEST_COUNTRY'], inplace = True)
    cleaned_data.drop(cleaned_data[cleaned_data['DEPARTURES_PERFORMED'] == 0].index, inplace = True)
    cleaned_data['DISTANCE'] = cleaned_data['DISTANCE'].apply(lambda x: x*0.868976) #convert miles to nautical miles

    cleaned_data['ds'] = cleaned_data['YEAR'].map(str) +"-"+ cleaned_data["MONTH"].map(str) + "-01"
    cleaned_data['ds'] = pd.to_datetime(cleaned_data['ds'],format='%Y-%m-%d')
    # cleaned_data.sort_values(by='ds', ascending=True, inplace = True)

    if regional_flag=='regional':
        #specific to regionality as mentioned in function desc
        cleaned_data = cleaned_data[(cleaned_data['DISTANCE'] <= 500) & (cleaned_data['DISTANCE'] >= 75)].reset_index()
        cleaned_data.drop(['index'], axis=1, inplace = True)

    if aircraft_flag=='nonjets':
        #specific to nonjets as mentioned in function desc
        cleaned_data = (cleaned_data[cleaned_data['AIRCRAFT_GROUP'] <= 5]).reset_index()
        cleaned_data.drop(['index'], axis=1, inplace = True)
        
    if aircraft_flag=='jets':
        #specific to nonjets as mentioned in function desc
        cleaned_data = (cleaned_data[cleaned_data['AIRCRAFT_GROUP'] >= 6]).reset_index()
        cleaned_data.drop(['index'], axis=1, inplace = True)    


    if analysis_type == 0:
        print(crt_year)
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US') & (cleaned_data['DEST_COUNTRY'] =='US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'US'


    elif analysis_type == 1:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US') & (cleaned_data['DEST_COUNTRY'] != 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'non-US'


    elif analysis_type == 2:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] != 'US') & (cleaned_data['DEST_COUNTRY'] == 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'non-US'
        cleaned_data['dest'] = 'US'


    elif analysis_type == 3:
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'all'


#     elif analysis_type == 4:
#         allstates = [None] * cleaned_data['ORIGIN_STATE_NM'].nunique()
#         allstates_index = 0
#         for crt_state in cleaned_data['ORIGIN_STATE_NM'].unique():
#             crtdata = cleaned_data[(cleaned_data['ORIGIN_STATE_NM'] == crt_state)].groupby('ds')['FREIGHT'].agg(['sum','count'])
#             crtdata.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
#             crtdata['origin'] = crt_state
#             crtdata['dest'] = 'all'
#             allstates[allstates_index] = crtdata
#             allstates_index=allstates_index+1

#         crtdata = pd.concat(allstates)
#         cleaned_data = crtdata.reset_index()
#         # cleaned_data.drop(['index'], axis=1, inplace = True)

    elif analysis_type == 4:
        cleaned_data= cleaned_data.groupby(['ds', 'ORIGIN_STATE_NM'])['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights', 'ORIGIN_STATE_NM': 'origin'}, inplace = True)
        cleaned_data['dest'] = 'all'
        
    elif analysis_type == 5:
        cleaned_data= cleaned_data.groupby(['ds', 'DEST_STATE_NM'])['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights', 'DEST_STATE_NM': 'dest'}, inplace = True)
        cleaned_data['origin'] = 'all'
        
    elif analysis_type == 6:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_COUNTRY'] == 'US')]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'US'
        cleaned_data['dest'] = 'all'
        
        
    elif analysis_type == 7:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 10194)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'AFW'
        cleaned_data['dest'] = 'all'
        
        
    elif analysis_type == 8:
        cleaned_data = cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 10194)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'AFW'
        
        
    elif analysis_type == 9:
        cleaned_data = cleaned_data[(cleaned_data['ORIGIN_AIRPORT_ID'] == 15601)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'KIS'
        cleaned_data['dest'] = 'all' 
        
    elif analysis_type == 10:
        cleaned_data = cleaned_data[(cleaned_data['DEST_AIRPORT_ID'] == 15601)]
        cleaned_data= cleaned_data.groupby('ds')['FREIGHT'].agg(['sum','count'])
        cleaned_data.reset_index(inplace=True)
        cleaned_data.rename(columns={'sum': 'y', 'count': 'num flights'}, inplace = True)
        cleaned_data['origin'] = 'all'
        cleaned_data['dest'] = 'KIS' 
      

    cleaned_data.to_csv('./../../../data/paav_cargo/agg_data/freight_aggregation__' +
                            str(crt_year) + '_analysis'+ str(analysis_type) + '_' + regional_flag+ '_' + aircraft_flag +
                        '.csv', mode='a', index=False, header=True)

In [ ]:
years_list = [yr for yr in range (1990,2024)]

for analysis_type in range(0,11):
    for regional_flag in [ 'alldistance' , 'regional']:
        for aircraft_flag in ['jetsandnonjets', 'nonjets','jets']:
            for crt_year in years_list:
                data_file = './../../../data/paav_cargo/agg_data/freight_aggregation__' + \
                                    str(crt_year) + '_analysis'+ str(analysis_type)+ '_' + regional_flag+ '_' + aircraft_flag +'.csv'
                if os.path.exists(data_file): os.remove(data_file), print("Deleted file " +data_file + ".")
                else: print("The file " +data_file + " does not exist")

                freight_aggregation(crt_year, 'FREIGHT', analysis_type, regional_flag, aircraft_flag)